# Домашняя работа по регуляризации и оптимизации

Ниже приводится корпус данных с двумя метками: 1 и -1. К данным применяется линейная модель классификации:

$f(x, \theta) = x_1 \theta_1 + x_2 \theta_2 + \theta_3.$

Предлагается подобрать параметры $\theta$ минимизируя следующую функцию ошибки:

$\mathcal{L}(\theta) = 0.1 \|\theta\|^2 + \frac{1}{N}\sum\limits_{i=1}^N \max(0, 1 - y_i f(x_i, \theta)).$

Для оптимизации предлагается использовать метод градиентного спуска с 1000 шагами размера $0.1$ из начальной точки $(1, 1, 0)$.

Набор данных \\

|    |   |   |   |      |      |
|----|---|---|---|------|------|
| X1 | 0 | 1 | 1 | -0.5 |  0   |
| X2 | 1 | 1 | 0 |  0.5 | -0.5 |
|  Y | 1 | 1 | 1 | -1   | -1   |

$ \theta_{start} = (1, 1, 0) $ 

In [ ]:
import numpy as np
import yaml

In [ ]:
X = np.array([
    [0, 1],
    [1, 1],
    [1, 0],
    [-0.5, 0.5],
    [0, -0.5]
])

y = np.array([1, 1, 1, -1, -1])

theta0 = np.array([1.0, 1.0, 0.0])

lr = 0.1

def f(X, theta):
    theta = np.asarray(theta)
    return (X * theta[:2]).sum(axis=-1) + theta[2]

def loss(X, y, theta):
    theta = np.asarray(theta)
    norm = (theta ** 2).sum()
    deltas = y * f(X, theta)
    return 0.1 * norm + np.mean(np.maximum(0, 1 - deltas))

print("Prediction:", f(X, theta0))
print("Loss:", loss(X, y, theta0))



deltas = np.array([0, 0.5, 0.64, 0.7, 0.7 ])
mask = np.ceil(deltas).astype(np.int64) #создаем маску из 
mask

Prediction: [ 1.   2.   1.   0.  -0.5]
Loss: 0.5


array([0, 1, 1, 1, 1])

Найдём производные функции линейной модели классификации и функции ошибки: \\
Производная функции линейной модели по $\theta$: \\
$  \frac{df(\theta)}{d\theta_k} = x_1 + x_2 + 1 $ \\
Производная функции ошибки $\theta$: \\
$\frac{d\mathcal{L}(\theta)}{d\theta_k} =  2*0.1*\theta_k + \frac{1}{N}\sum\limits_{i=1}^N\begin{cases}
  0 & \text{if  }  (1 - y_i f(x_i, \theta)) \leq 0 \\
  -y_i * \frac{\mathrm d}{\mathrm d \theta_k}f(x_i, \theta)& \text{else}
\end{cases}$



In [ ]:
# Ваш код оптимизации.
def diff_f(X):
  X_t = X.transpose()
  return np.array([X_t[0], X_t[1], [1] * len(X_t[0])])

def diff_loss(X, y, theta):
  theta = np.asarray(theta)


  norm = 2*0.1*theta
  
  deltas = np.maximum(0, 1 - y*f(X, theta)) #находим значения лежащие в [0; 1], то есть те, которые подходят под условие if(1−y_i*f(x_i,θ))≤0)
  mask = np.ceil(deltas).astype(np.int64) #создаем маску из ненулевых значений

  deltas_diff = mask*((-1)*y*diff_f(X)) #применяем маску к производным deltas
  return norm + np.mean(deltas_diff, axis=1)

theta = theta0
X_t = X.transpose()



In [ ]:
theta = theta0
for step in range(10000):
  theta = theta - lr * diff_loss(X, y, theta)

In [ ]:
print("Prediction:", f(X, theta))
print("Loss:", loss(X, y, theta))

with open("submission.yaml", "w") as fp:
    yaml.safe_dump({"tasks": [{"task1": {"answer": theta.tolist()}}]}, fp)

Prediction: [ 0.74994308  1.99988615  0.99988615 -0.37502846 -0.75005692]
Loss: 0.48750569295176926
